<a href="https://colab.research.google.com/github/nilesh3030/Stroke-Prediction/blob/main/MLflow_Implementation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)

model_data = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/model_data.csv', low_memory = False)

In [ ]:
model_data = model_data.drop(['Unnamed: 0'], axis = 1)

In [ ]:
model_data.head()

In [ ]:
!pip install mlflow

In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [ ]:
def eval_metrics(actual, pred):
    accuracy = accuracy_score(actual, pred)
    class_report = classification_report(actual, pred)
    return accuracy, class_report

In [ ]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    X = model_data.iloc[:,model_data.columns != 'cv2_cphss']
    y = model_data.cv2_cphss
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.20, random_state=5, stratify=y)

    with mlflow.start_run():
        lr = LogisticRegression()
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (accuracy, class_report) = eval_metrics(test_y, predicted_qualities)

        print("  accuracy: %s" % accuracy)
        print("  class_report: %s" % class_report)

        mlflow.log_metric("accuracy", accuracy)
        #mlflow.log_metric("class_report", class_report)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="test_LR")
        else:
            mlflow.sklearn.log_model(lr, "model")

In [ ]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    X = model_data.iloc[:,model_data.columns != 'cv2_cphss']
    y = model_data.cv2_cphss
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5, stratify=y)

    with mlflow.start_run():
        from sklearn.ensemble import RandomForestClassifier

        # creating a RF classifier
        clf = RandomForestClassifier(n_estimators = 100, max_features = 'sqrt', max_depth = 5, min_samples_leaf = 3, min_samples_split = 5) 
        
        # Training the model on the training dataset
        # fit function is used to train the model using the training sets as parameters
        clf.fit(X_train, y_train)
        
        # performing predictions on the test dataset
        y_pred = clf.predict(X_test)

        # metrics are used to find accuracy or error
        train_acc = clf.score(X_train, y_train)
        print("The Accuracy for Training Set is {}".format(train_acc*100))

        test_acc = accuracy_score(y_test, y_pred)
        print("The Accuracy for Test Set is {}".format(test_acc*100))

        print(classification_report(y_test, y_pred))

        mlflow.log_metric("accuracy", test_acc)
        #mlflow.log_metric("class_report", class_report)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="test_LR")
        else:
            mlflow.sklearn.log_model(lr, "model")